In [1]:
import os
import datetime
import random
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, Band
from bokeh.embed import components
from bokeh.models import Range1d
from bokeh.models import NumeralTickFormatter


import numpy as np

In [2]:
import pandas as pd


In [3]:
import statsmodels.api as sm
import math

In [7]:
import math
output_notebook()
def get_data(state):
    df = pd.read_csv('data/states.csv')
    df = df[(df['state'] == state)]
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by = ['date'])
    return df

def get_data_us():
    df = pd.read_csv('data/world.csv')
    df = df[df['country'] == 'United_States_of_America']
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by = ['date'])
    return df

def get_data_us2():
    df = pd.read_csv('data/us.csv')
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by = ['date'])
    return df

def filter_neg(x, y):
    x_, y_ = [], []
    for counter, i in enumerate(x):
        if i < 0 or y[counter] < 0:
            continue
        x_.append(i)
        y_.append( y[counter])
    return x_, y_

def make_p(deaths, cases, state = None):
    zz = list(zip(cases, deaths))
    zz = sorted(zz)
    x = [x[0] for x in zz]
    y = [x[1] for x in zz]
    x, y = filter_neg(x, y)
    X = list(zip(*[x]))
    xm = sm.add_constant(X)
    model = sm.OLS(y, xm) 
    result = model.fit()
    y_hat = result.predict(xm)
    #print(result.summary())
    format_intercept = '{0:,d}'.format(int(round(result.params[0], 2)))
    p = figure(title = '{state} {i} + {s}x'.format(state = state, i = format_intercept,
       s = round(result.params[1],3)))
    p.xaxis[0].formatter = NumeralTickFormatter(format="0,")
    p.yaxis[0].formatter = NumeralTickFormatter(format="0,")


    p.line(x = x, y = y_hat)
    p.circle(x = x, y = y)
    return result.rsquared, p, result.params[0], result.params[1]
    
def make_rel(state = None, min_date = datetime.datetime(2020,8,1),
            move = 21):
    #ifr is 1.15% = .011
    if state =='US':
        df = get_data_us()
    else:
        df = get_data(state = state)
    df = df[df['date'] >= min_date]
    dates = df['date'].tolist()
    cases = df['cases'].tolist()
    deaths = df['deaths'].tolist()
    d_ori = {}
    for counter, i in enumerate(dates):
        d_ori[i] = (deaths[counter], cases[counter])
    def move_deaths(move = move):
        d = {}
        for i in dates:
            new_date = i + datetime.timedelta(days = move)
            death_ = d_ori.get(new_date)
            if not death_:
                break
            death = death_[0]
            d[i] = (death, death_[1])
        return d
    moved_d = move_deaths(7)
    deaths_l, cases_l = [], []
    for key in moved_d.keys():
        deaths_l.append(moved_d[key][0])
        cases_l.append(moved_d[key][1])
    r, p, intercept, slope = make_p(deaths_l, cases_l, state = state)
    return p
            
            
show(make_rel(state = 'US', min_date = datetime.datetime(2020,8,1)))

Loading BokehJS ...